In [1]:
# 필요한 라이브러리 로드
from pyspark.sql import SparkSession

In [2]:
# Redshift JDBC 드라이버 JAR 파일 경로
driver_path = "/Users/admin/Desktop/teamproject/softeer-team-project/EDA/wh/red_shift_demo/redshift-jdbc42-2.1.0.29/redshift-jdbc42-2.1.0.29.jar"

# SparkSession 생성 시 JDBC 드라이버 추가
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars", driver_path) \
    .getOrCreate()

24/08/19 18:57:44 WARN Utils: Your hostname, adminui-MacBookPro-2.local resolves to a loopback address: 127.0.0.1; using 10.31.170.63 instead (on interface en0)
24/08/19 18:57:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/08/19 18:57:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import json

# JSON 파일 경로
json_file_path = 'properties.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as file:
    connection_properties = json.load(file)
    
# 예제 사용
jdbc_url = connection_properties["jdbc_url"]
user = connection_properties["user"]
password = connection_properties["password"]
driver = connection_properties["driver"]



In [4]:
# 예제: model_result.csv 로드 이미 index가 있는 경우
data_df = spark.read.csv('model_result.csv', header=True, inferSchema=True) 

# 0번 컬럼을 drop
data_df = data_df.drop('_c0')

# 5개의 행만 남기고 나머지는 제거
data_df = data_df.limit(5)

In [5]:
data_df.show()

+-----+-------------------+--------------+---------+-------------+
|index|                pdf|cumulative_num|post_type|relative_time|
+-----+-------------------+--------------+---------+-------------+
|    0|0.08452332244990605|             0|        1|            0|
|    1|0.11541961535467207|             1|        1|            0|
|    2| 0.1381065214831182|             2|        1|            0|
|    3|0.14480389780038194|             3|        1|            0|
|    4|0.13303864692462333|             4|        1|            0|
+-----+-------------------+--------------+---------+-------------+



In [6]:
# DataFrame을 Redshift 테이블로 저장
data_df.write.jdbc(
    url=jdbc_url,
    table="MODEL",
    mode="overwrite",  # "overwrite" 또는 "append"
    properties=connection_properties
)



24/08/19 18:57:53 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8


24/08/19 18:57:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
# conection close
spark.stop()